<a href="https://colab.research.google.com/github/belerico/aml-project/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow==2.0 keras==2.3.1

  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 378kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 8.8MB/s 
     |████████████████████████████████| 450kB 19.0MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1

In [0]:
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/AML')

# REPO OPERATIONS
If you want to clone the repo

In [0]:
# Clone the entire repo in current directory
!git clone -l -s https://github.com/belerico/aml-project.git aml-project
%cd aml-project
!ls

Cloning into 'aml-project'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 49 (delta 18), reused 40 (delta 13), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/drive/My Drive/AML/aml-project
dataset  LICENSE  main.py  models.py  README.md  references  test  utils.py


In [0]:
 ! git config  user.email "gianluca@scarpellini.dev"
 ! git config user.name "gianscarpe"

If you want to push

In [10]:
! git add main.ipynb
! git commit -m 'Colab main for experiments'
! git push


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@7ed519867ad1.(none)')
fatal: could not read Username for 'https://github.com': No such device or address


fatal: could not read Username for 'https://github.com': No such device or address


# Execution


In [3]:
%cd aml-project/


/content/drive/My Drive/AML/aml-project


In [0]:
from dataset import get_data
from models import get_deep_cross_model
from fitness import fit
from utils import dot_similarity, get_pretrained_embedding
import json

In [0]:
config = {
    'DATA_PATH' : 'data/all_train_xlarge.json',
    'EXP_DIR' : 'data/exps/exp1',
    'PRETRAINED_EMBEDDING_PATH' : '',
    'NUM_WORDS' : 1000,
    'MAX_LEN' : 20,
    'BATCH_SIZE' : 128,
    'EMBEDDING_DIM' : 100,
    'EARLY_STOPPING' : 10
}

In [0]:
print('Loading data')
train_gen, val_gen, word_index = get_data(DATA_PATH, NUM_WORDS, MAX_LEN, BATCH_SIZE,
                                          train_test_split=0.8)
print('Data loaded in RAM')

Loading data
Data loaded in RAM


In [0]:
matrix_similarity_function = dot_similarity
embedding_matrix = None

if PRETRAINED_EMBEDDING_PATH != "":
    embedding_matrix = get_pretrained_embedding(PRETRAINED_EMBEDDING_PATH, NUM_WORDS + 1,
                                                EMBEDDING_DIM, word_index)

model = fit(train_gen, val_gen, NUM_WORDS, EMBEDDING_DIM, MAX_LEN,
                matrix_similarity_function,EXP_DIR,
                EARLY_STOPPING,
                embedding_matrix=None)


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 20, 100)      100100      input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 20, 200)      160800      embedding_5[0][0]          

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1342/1342 [==============================] - 405s 302ms/step - loss: 0.0472 - accuracy: 0.9825 - val_loss: 2.4727 - val_accuracy: 0.8037
Epoch 2/10
1342/1342 [==============================] - 395s 295ms/step - loss: 0.0470 - accuracy: 0.9828 - val_loss: 1.7176 - val_accuracy: 0.7925
Epoch 3/10
1342/1342 [==============================] - 392s 292ms/step - loss: 0.0418 - accuracy: 0.9846 - val_loss: 2.7975 - val_accuracy: 0.7928
Epoch 4/10
1342/1342 [==============================] - 387s 289ms/step - loss: 0.0344 - accuracy: 0.9872 - val_loss: 1.6244 - val_accuracy: 0.8352
Epoch 5/10
1342/1342 [==============================] - 396s 295ms/step - loss: 0.0313 - accuracy: 0.9879 - val_loss: 1.4512 - val_accuracy: 0.8375
Epoch 6/10
1342/1342 [==============================] - 391s 291ms/step - loss: 0.0241 - accuracy: 0.9909 - val_loss: 2.1697 - val_accuracy: 0.7734
Epoch 7/10
1342/1342 [==============================] - 392s 292ms/step - loss: 0.0220 - accuracy: 0.9925 - val_

In [0]:
config_file = os.path.join(config['EXP_DIR'], 'config.json')
with open(config_file, 'w') as json_file:
  data = json.dumps(config)
  json_file.write(data)